### 1. 발전 실적을 시간 컬럼 하나에 다 때려박기 (원본에 있던 거)

In [ ]:
import pandas as pd
import numpy as np

Mounted at /content/gdrive


In [ ]:
data=pd.read_csv('./windPowerData/time_power.csv', header=None) # 컬럼명 숫자로 들어가게
data.head(25)

,0,1,2,3,4,5,6,7,8,9,...,2515,2516,2517,2518,2519,2520,2521,2522,2523,2524
0,0,3031,5741,8300,7822,253,7400,9893,11435,7575,...,1180,1573,680,0,9998,6657,1527,4589,1385,341
1,0,3051,8666,8573,8462,1593,9963,9542,11982,7607,...,1056,1230,933,0,9992,6274,1234,4392,1504,114
2,92,3173,10320,8584,8129,4009,7356,10309,9841,8289,...,1026,583,821,113,9330,6278,884,4071,2369,105
3,269,4196,10799,7192,9295,6941,6366,8454,11463,5521,...,761,303,844,0,6878,6605,748,4389,3027,64
4,541,5429,8959,7834,10876,8202,9109,5602,9826,5429,...,881,26,380,342,5994,4342,1247,5753,4530,0
5,945,7473,9078,7959,10816,6029,10043,5768,7631,6637,...,1117,53,308,396,6078,3008,2001,5555,3452,0
6,699,4464,10585,8930,10215,4359,9592,6056,7508,6463,...,1404,414,564,953,7730,3906,2635,3558,4203,0
7,1371,2120,8188,8961,10735,1817,10755,6048,6314,4360,...,1104,276,1535,497,9981,2661,3065,5444,4916,0
8,1974,1935,8642,10024,10172,524,9239,7667,5859,2973,...,649,220,1975,535,9621,2323,3647,5975,3210,0
9,1425,2798,10433,10800,9078,398,8869,6399,5105,2549,...,1074,303,1611,127,9583,1069,3339,6590,2052,0


In [ ]:
type(data)

pandas.core.frame.DataFrame

In [ ]:
test = pd.DataFrame(columns=['windForce'])

In [ ]:
type(test)
print(test.shape)
print(test)

(0, 1)
Empty DataFrame
Columns: [windForce]
Index: []


In [ ]:
data[1][2]

3173

In [ ]:
for i in range(0,len(data.columns)):
    for j in range(0,24): # 마음에 들지 않지만 하나하나 골라서 추가,,,
        test = test.append({"windForce":data[i][j]}, ignore_index=True)
        print(i,j)

0 0
0 1
0 2
0 3
0 4
0 5
0 6
0 7
0 8
0 9
0 10
0 11
0 12
0 13
0 14
0 15
0 16
0 17
0 18
0 19
0 20
0 21
0 22
0 23
1 0
1 1
1 2
1 3
1 4
1 5
1 6
1 7
1 8
1 9
1 10
1 11
1 12
1 13
1 14
1 15
1 16
1 17
1 18
1 19
1 20
1 21
1 22
1 23
2 0
2 1
2 2
2 3
2 4
2 5
2 6
2 7
2 8
2 9
2 10
2 11
2 12
2 13
2 14
2 15
2 16
2 17
2 18
2 19
2 20
2 21
2 22
2 23
3 0
3 1
3 2
3 3
3 4
3 5
3 6
3 7
3 8
3 9
3 10
3 11
3 12
3 13
3 14
3 15
3 16
3 17
3 18
3 19
3 20
3 21
3 22
3 23
4 0
4 1
4 2
4 3
4 4
4 5
4 6
4 7
4 8
4 9
4 10
4 11
4 12
4 13
4 14
4 15
4 16
4 17
4 18
4 19
4 20
4 21
4 22
4 23
5 0
5 1
5 2
5 3
5 4
5 5
5 6
5 7
5 8
5 9
5 10
5 11
5 12
5 13
5 14
5 15
5 16
5 17
5 18
5 19
5 20
5 21
5 22
5 23
6 0
6 1
6 2
6 3
6 4
6 5
6 6
6 7
6 8
6 9
6 10
6 11
6 12
6 13
6 14
6 15
6 16
6 17
6 18
6 19
6 20
6 21
6 22
6 23
7 0
7 1
7 2
7 3
7 4
7 5
7 6
7 7
7 8
7 9
7 10
7 11
7 12
7 13
7 14
7 15
7 16
7 17
7 18
7 19
7 20
7 21
7 22
7 23
8 0
8 1
8 2
8 3
8 4
8 5
8 6
8 7
8 8
8 9
8 10
8 11
8 12
8 13
8 14
8 15
8 16
8 17
8 18
8 19
8 20
8 21
8 22
8 23
9 0
9 1
9 

67 23
68 0
68 1
68 2
68 3
68 4
68 5
68 6
68 7
68 8
68 9
68 10
68 11
68 12
68 13
68 14
68 15
68 16
68 17
68 18
68 19
68 20
68 21
68 22
68 23
69 0
69 1
69 2
69 3
69 4
69 5
69 6
69 7
69 8
69 9
69 10
69 11
69 12
69 13
69 14
69 15
69 16
69 17
69 18
69 19
69 20
69 21
69 22
69 23
70 0
70 1
70 2
70 3
70 4
70 5
70 6
70 7
70 8
70 9
70 10
70 11
70 12
70 13
70 14
70 15
70 16
70 17
70 18
70 19
70 20
70 21
70 22
70 23
71 0
71 1
71 2
71 3
71 4
71 5
71 6
71 7
71 8
71 9
71 10
71 11
71 12
71 13
71 14
71 15
71 16
71 17
71 18
71 19
71 20
71 21
71 22
71 23
72 0
72 1
72 2
72 3
72 4
72 5
72 6
72 7
72 8
72 9
72 10
72 11
72 12
72 13
72 14
72 15
72 16
72 17
72 18
72 19
72 20
72 21
72 22
72 23
73 0
73 1
73 2
73 3
73 4
73 5
73 6
73 7
73 8
73 9
73 10
73 11
73 12
73 13
73 14
73 15
73 16
73 17
73 18
73 19
73 20
73 21
73 22
73 23
74 0
74 1
74 2
74 3
74 4
74 5
74 6
74 7
74 8
74 9
74 10
74 11
74 12
74 13
74 14
74 15
74 16
74 17
74 18
74 19
74 20
74 21
74 22
74 23
75 0
75 1
75 2
75 3
75 4
75 5
75 6
75 7
75 8
75 9
75 10


130 2
130 3
130 4
130 5
130 6
130 7
130 8
130 9
130 10
130 11
130 12
130 13
130 14
130 15
130 16
130 17
130 18
130 19
130 20
130 21
130 22
130 23
131 0
131 1
131 2
131 3
131 4
131 5
131 6
131 7
131 8
131 9
131 10
131 11
131 12
131 13
131 14
131 15
131 16
131 17
131 18
131 19
131 20
131 21
131 22
131 23
132 0
132 1
132 2
132 3
132 4
132 5
132 6
132 7
132 8
132 9
132 10
132 11
132 12
132 13
132 14
132 15
132 16
132 17
132 18
132 19
132 20
132 21
132 22
132 23
133 0
133 1
133 2
133 3
133 4
133 5
133 6
133 7
133 8
133 9
133 10
133 11
133 12
133 13
133 14
133 15
133 16
133 17
133 18
133 19
133 20
133 21
133 22
133 23
134 0
134 1
134 2
134 3
134 4
134 5
134 6
134 7
134 8
134 9
134 10
134 11
134 12
134 13
134 14
134 15
134 16
134 17
134 18
134 19
134 20
134 21
134 22
134 23
135 0
135 1
135 2
135 3
135 4
135 5
135 6
135 7
135 8
135 9
135 10
135 11
135 12
135 13
135 14
135 15
135 16
135 17
135 18
135 19
135 20
135 21
135 22
135 23
136 0
136 1
136 2
136 3
136 4
136 5
136 6
136 7
136 8
136 9
136 

187 3
187 4
187 5
187 6
187 7
187 8
187 9
187 10
187 11
187 12
187 13
187 14
187 15
187 16
187 17
187 18
187 19
187 20
187 21
187 22
187 23
188 0
188 1
188 2
188 3
188 4
188 5
188 6
188 7
188 8
188 9
188 10
188 11
188 12
188 13
188 14
188 15
188 16
188 17
188 18
188 19
188 20
188 21
188 22
188 23
189 0
189 1
189 2
189 3
189 4
189 5
189 6
189 7
189 8
189 9
189 10
189 11
189 12
189 13
189 14
189 15
189 16
189 17
189 18
189 19
189 20
189 21
189 22
189 23
190 0
190 1
190 2
190 3
190 4
190 5
190 6
190 7
190 8
190 9
190 10
190 11
190 12
190 13
190 14
190 15
190 16
190 17
190 18
190 19
190 20
190 21
190 22
190 23
191 0
191 1
191 2
191 3
191 4
191 5
191 6
191 7
191 8
191 9
191 10
191 11
191 12
191 13
191 14
191 15
191 16
191 17
191 18
191 19
191 20
191 21
191 22
191 23
192 0
192 1
192 2
192 3
192 4
192 5
192 6
192 7
192 8
192 9
192 10
192 11
192 12
192 13
192 14
192 15
192 16
192 17
192 18
192 19
192 20
192 21
192 22
192 23
193 0
193 1
193 2
193 3
193 4
193 5
193 6
193 7
193 8
193 9
193 10
193

241 10
241 11
241 12
241 13
241 14
241 15
241 16
241 17
241 18
241 19
241 20
241 21
241 22
241 23
242 0
242 1
242 2
242 3
242 4
242 5
242 6
242 7
242 8
242 9
242 10
242 11
242 12
242 13
242 14
242 15
242 16
242 17
242 18
242 19
242 20
242 21
242 22
242 23
243 0
243 1
243 2
243 3
243 4
243 5
243 6
243 7
243 8
243 9
243 10
243 11
243 12
243 13
243 14
243 15
243 16
243 17
243 18
243 19
243 20
243 21
243 22
243 23
244 0
244 1
244 2
244 3
244 4
244 5
244 6
244 7
244 8
244 9
244 10
244 11
244 12
244 13
244 14
244 15
244 16
244 17
244 18
244 19
244 20
244 21
244 22
244 23
245 0
245 1
245 2
245 3
245 4
245 5
245 6
245 7
245 8
245 9
245 10
245 11
245 12
245 13
245 14
245 15
245 16
245 17
245 18
245 19
245 20
245 21
245 22
245 23
246 0
246 1
246 2
246 3
246 4
246 5
246 6
246 7
246 8
246 9
246 10
246 11
246 12
246 13
246 14
246 15
246 16
246 17
246 18
246 19
246 20
246 21
246 22
246 23
247 0
247 1
247 2
247 3
247 4
247 5
247 6
247 7
247 8
247 9
247 10
247 11
247 12
247 13
247 14
247 15
247 16
247

...

2329 14
2329 15
2329 16
2329 17
2329 18
2329 19
2329 20
2329 21
2329 22
2329 23
2330 0
2330 1
2330 2
2330 3
2330 4
2330 5
2330 6
2330 7
2330 8
2330 9
2330 10
2330 11
2330 12
2330 13
2330 14
2330 15
2330 16
2330 17
2330 18
2330 19
2330 20
2330 21
2330 22
2330 23
2331 0
2331 1
2331 2
2331 3
2331 4
2331 5
2331 6
2331 7
2331 8
2331 9
2331 10
2331 11
2331 12
2331 13
2331 14
2331 15
2331 16
2331 17
2331 18
2331 19
2331 20
2331 21
2331 22
2331 23
2332 0
2332 1
2332 2
2332 3
2332 4
2332 5
2332 6
2332 7
2332 8
2332 9
2332 10
2332 11
2332 12
2332 13
2332 14
2332 15
2332 16
2332 17
2332 18
2332 19
2332 20
2332 21
2332 22
2332 23
2333 0
2333 1
2333 2
2333 3
2333 4
2333 5
2333 6
2333 7
2333 8
2333 9
2333 10
2333 11
2333 12
2333 13
2333 14
2333 15
2333 16
2333 17
2333 18
2333 19
2333 20
2333 21
2333 22
2333 23
2334 0
2334 1
2334 2
2334 3
2334 4
2334 5
2334 6
2334 7
2334 8
2334 9
2334 10
2334 11
2334 12
2334 13
2334 14
2334 15
2334 16
2334 17
2334 18
2334 19
2334 20
2334 21
2334 22
2334 23
2335 0
233

2374 20
2374 21
2374 22
2374 23
2375 0
2375 1
2375 2
2375 3
2375 4
2375 5
2375 6
2375 7
2375 8
2375 9
2375 10
2375 11
2375 12
2375 13
2375 14
2375 15
2375 16
2375 17
2375 18
2375 19
2375 20
2375 21
2375 22
2375 23
2376 0
2376 1
2376 2
2376 3
2376 4
2376 5
2376 6
2376 7
2376 8
2376 9
2376 10
2376 11
2376 12
2376 13
2376 14
2376 15
2376 16
2376 17
2376 18
2376 19
2376 20
2376 21
2376 22
2376 23
2377 0
2377 1
2377 2
2377 3
2377 4
2377 5
2377 6
2377 7
2377 8
2377 9
2377 10
2377 11
2377 12
2377 13
2377 14
2377 15
2377 16
2377 17
2377 18
2377 19
2377 20
2377 21
2377 22
2377 23
2378 0
2378 1
2378 2
2378 3
2378 4
2378 5
2378 6
2378 7
2378 8
2378 9
2378 10
2378 11
2378 12
2378 13
2378 14
2378 15
2378 16
2378 17
2378 18
2378 19
2378 20
2378 21
2378 22
2378 23
2379 0
2379 1
2379 2
2379 3
2379 4
2379 5
2379 6
2379 7
2379 8
2379 9
2379 10
2379 11
2379 12
2379 13
2379 14
2379 15
2379 16
2379 17
2379 18
2379 19
2379 20
2379 21
2379 22
2379 23
2380 0
2380 1
2380 2
2380 3
2380 4
2380 5
2380 6
2380 7
23

2422 9
2422 10
2422 11
2422 12
2422 13
2422 14
2422 15
2422 16
2422 17
2422 18
2422 19
2422 20
2422 21
2422 22
2422 23
2423 0
2423 1
2423 2
2423 3
2423 4
2423 5
2423 6
2423 7
2423 8
2423 9
2423 10
2423 11
2423 12
2423 13
2423 14
2423 15
2423 16
2423 17
2423 18
2423 19
2423 20
2423 21
2423 22
2423 23
2424 0
2424 1
2424 2
2424 3
2424 4
2424 5
2424 6
2424 7
2424 8
2424 9
2424 10
2424 11
2424 12
2424 13
2424 14
2424 15
2424 16
2424 17
2424 18
2424 19
2424 20
2424 21
2424 22
2424 23
2425 0
2425 1
2425 2
2425 3
2425 4
2425 5
2425 6
2425 7
2425 8
2425 9
2425 10
2425 11
2425 12
2425 13
2425 14
2425 15
2425 16
2425 17
2425 18
2425 19
2425 20
2425 21
2425 22
2425 23
2426 0
2426 1
2426 2
2426 3
2426 4
2426 5
2426 6
2426 7
2426 8
2426 9
2426 10
2426 11
2426 12
2426 13
2426 14
2426 15
2426 16
2426 17
2426 18
2426 19
2426 20
2426 21
2426 22
2426 23
2427 0
2427 1
2427 2
2427 3
2427 4
2427 5
2427 6
2427 7
2427 8
2427 9
2427 10
2427 11
2427 12
2427 13
2427 14
2427 15
2427 16
2427 17
2427 18
2427 19
242

2470 23
2471 0
2471 1
2471 2
2471 3
2471 4
2471 5
2471 6
2471 7
2471 8
2471 9
2471 10
2471 11
2471 12
2471 13
2471 14
2471 15
2471 16
2471 17
2471 18
2471 19
2471 20
2471 21
2471 22
2471 23
2472 0
2472 1
2472 2
2472 3
2472 4
2472 5
2472 6
2472 7
2472 8
2472 9
2472 10
2472 11
2472 12
2472 13
2472 14
2472 15
2472 16
2472 17
2472 18
2472 19
2472 20
2472 21
2472 22
2472 23
2473 0
2473 1
2473 2
2473 3
2473 4
2473 5
2473 6
2473 7
2473 8
2473 9
2473 10
2473 11
2473 12
2473 13
2473 14
2473 15
2473 16
2473 17
2473 18
2473 19
2473 20
2473 21
2473 22
2473 23
2474 0
2474 1
2474 2
2474 3
2474 4
2474 5
2474 6
2474 7
2474 8
2474 9
2474 10
2474 11
2474 12
2474 13
2474 14
2474 15
2474 16
2474 17
2474 18
2474 19
2474 20
2474 21
2474 22
2474 23
2475 0
2475 1
2475 2
2475 3
2475 4
2475 5
2475 6
2475 7
2475 8
2475 9
2475 10
2475 11
2475 12
2475 13
2475 14
2475 15
2475 16
2475 17
2475 18
2475 19
2475 20
2475 21
2475 22
2475 23
2476 0
2476 1
2476 2
2476 3
2476 4
2476 5
2476 6
2476 7
2476 8
2476 9
2476 10
2476

2517 10
2517 11
2517 12
2517 13
2517 14
2517 15
2517 16
2517 17
2517 18
2517 19
2517 20
2517 21
2517 22
2517 23
2518 0
2518 1
2518 2
2518 3
2518 4
2518 5
2518 6
2518 7
2518 8
2518 9
2518 10
2518 11
2518 12
2518 13
2518 14
2518 15
2518 16
2518 17
2518 18
2518 19
2518 20
2518 21
2518 22
2518 23
2519 0
2519 1
2519 2
2519 3
2519 4
2519 5
2519 6
2519 7
2519 8
2519 9
2519 10
2519 11
2519 12
2519 13
2519 14
2519 15
2519 16
2519 17
2519 18
2519 19
2519 20
2519 21
2519 22
2519 23
2520 0
2520 1
2520 2
2520 3
2520 4
2520 5
2520 6
2520 7
2520 8
2520 9
2520 10
2520 11
2520 12
2520 13
2520 14
2520 15
2520 16
2520 17
2520 18
2520 19
2520 20
2520 21
2520 22
2520 23
2521 0
2521 1
2521 2
2521 3
2521 4
2521 5
2521 6
2521 7
2521 8
2521 9
2521 10
2521 11
2521 12
2521 13
2521 14
2521 15
2521 16
2521 17
2521 18
2521 19
2521 20
2521 21
2521 22
2521 23
2522 0
2522 1
2522 2
2522 3
2522 4
2522 5
2522 6
2522 7
2522 8
2522 9
2522 10
2522 11
2522 12
2522 13
2522 14
2522 15
2522 16
2522 17
2522 18
2522 19
2522 20
25

In [ ]:
test.shape

(60600, 1)

In [ ]:
test.to_csv("time_power_1_row.csv", mode='w')

### 2.기상청 데이터 이어 붙이기

In [ ]:
data1=pd.read_csv('./windPowerData/forecast/1.csv', encoding='euc-kr', header = None)

data1 = data1.fillna(0) # 빈 값 0으로 
data1 = data1.drop(index=0) # 컬럼명 적혀있는 거 지움
data1 = data1.drop(0,1) # 컬럼 삭제는 axis = 1 그래서 뒤에 1 있는 거
data1 = data1.drop(1,1) # 지점명 컬럼 삭제
data1 = data1.reset_index()  #index reset
data1 = data1.drop('index',1) # index reset 하면 기존 index가 컬럼에 추가되므로 삭제
data1.columns  = ['date','tem','temQC'
          ,'rain','rainQC'
           ,'windSpeed','windSpeedQC','windDirection','windDirectionQC'
           ,'snow','snow3Hour'
           ,'cloud','middleCloud','clodeUn','highCloud'] #컬럼명 지정

data1.head(5)

,date,tem,temQC,rain,rainQC,windSpeed,windSpeedQC,windDirection,windDirectionQC,snow,snow3Hour,cloud,middleCloud,clodeUn,highCloud
0,2013-01-01 00:00,1.3,0,0,9,3.2,0,340,0,0,0,3,3,Sc,10
1,2013-01-01 01:00,.9,0,0,0,2.6,0,320,0,0,0,0,0,0,0
2,2013-01-01 02:00,1.5,0,0,0,3,0,340,0,0,0,0,0,0,0
3,2013-01-01 03:00,2.7,0,0,0,1.7,0,270,0,0,0,9,9,Sc,10
4,2013-01-01 04:00,.6,0,0,9,1.8,0,320,0,0,0,3,3,Sc,10


In [ ]:
len(data1.index) #row 수

4344

In [ ]:
# test1 = data1
# test1 = test1.reset_index()
# test1 = test1.drop('index',1)
# test1.columns  = ['date','tem','temQC'
#        ,'rain','rainQC'
#        ,'windSpeed','windSpeedQC','windDirection','windDirectionQC'
#        ,'snow','snow3Hour'
#        ,'cloud','middleCloud','clodeUn','highCloud']
# test1

,date,tem,temQC,rain,rainQC,windSpeed,windSpeedQC,windDirection,windDirectionQC,snow,snow3Hour,cloud,middleCloud,clodeUn,highCloud
0,2013-01-01 00:00,1.3,0.0,0.0,9.0,3.2,0.0,340.0,0.0,0.0,0.0,3.0,3.0,Sc,10.0
1,2013-01-01 01:00,0.9,0.0,0.0,0.0,2.6,0.0,320.0,0.0,0.0,0.0,0.0,0.0,0,0.0
2,2013-01-01 02:00,1.5,0.0,0.0,0.0,3.0,0.0,340.0,0.0,0.0,0.0,0.0,0.0,0,0.0
3,2013-01-01 03:00,2.7,0.0,0.0,0.0,1.7,0.0,270.0,0.0,0.0,0.0,9.0,9.0,Sc,10.0
4,2013-01-01 04:00,0.6,0.0,0.0,9.0,1.8,0.0,320.0,0.0,0.0,0.0,3.0,3.0,Sc,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4339,2013-06-30 19:00,23.4,0.0,0.0,0.0,1.4,0.0,50.0,0.0,0.0,0.0,0.0,0.0,0,0.0
4340,2013-06-30 20:00,23.2,0.0,0.0,0.0,1.0,0.0,50.0,0.0,0.0,0.0,0.0,0.0,0,0.0
4341,2013-06-30 21:00,22.9,0.0,0.0,0.0,0.6,0.0,340.0,0.0,0.0,0.0,7.0,7.0,Sc,10.0
4342,2013-06-30 22:00,22.4,0.0,0.0,0.0,0.6,0.0,340.0,0.0,0.0,0.0,0.0,0.0,0,0.0


In [ ]:
for i in range(2,15):
    fileName = './windPowerData/forecast/'+str(i)+'.csv'
    data2 = pd.read_csv(fileName, encoding='euc-kr', header = None)# 컬럼명 숫자로 들어가게
    data2 = data2.fillna(0) # 빈 값 0으로 
    data2 = data2.drop(index=0) # 컬럼명 적혀있는 거 지움
    data2 = data2.drop(0,1) # 지점 컬럼 삭제는 axis = 1 그래서 뒤에 1 있는 거
    data2 = data2.drop(1,1) # 지점명 컬럼 삭제
    data2 = data2.reset_index()  #index reset
    data2 = data2.drop('index',1) # index reset 하면 기존 index가 컬럼에 추가되므로 삭제
    data2.columns  = ['date','tem','temQC'
           ,'rain','rainQC'
           ,'windSpeed','windSpeedQC','windDirection','windDirectionQC'
           ,'snow','snow3Hour'
           ,'cloud','middleCloud','clodeUn','highCloud'] #컬럼명 지정
    
    data1 = pd.concat([data1, data2]) # 아래로 데이터 합치기(df 끼리, 아주 효자임)
data1.to_csv("weather_data_day.csv", mode='w')

In [ ]:
data1

,date,tem,temQC,rain,rainQC,windSpeed,windSpeedQC,windDirection,windDirectionQC,snow,snow3Hour,cloud,middleCloud,clodeUn,highCloud
0,2013-01-01 00:00,1.3,0,0,9,3.2,0,340,0,0,0,3,3,Sc,10
1,2013-01-01 01:00,.9,0,0,0,2.6,0,320,0,0,0,0,0,0,0
2,2013-01-01 02:00,1.5,0,0,0,3,0,340,0,0,0,0,0,0,0
3,2013-01-01 03:00,2.7,0,0,0,1.7,0,270,0,0,0,9,9,Sc,10
4,2013-01-01 04:00,.6,0,0,9,1.8,0,320,0,0,0,3,3,Sc,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3667,2019-11-30 19:00,14.1,0,0,0,2.5,0,50,0,0,0,9,4,0,22
3668,2019-11-30 20:00,14.1,0,0,0,2.3,0,50,0,0,0,8,8,0,22
3669,2019-11-30 21:00,13.3,0,0,0,3.6,0,360,0,0,0,10,10,0,21
3670,2019-11-30 22:00,12.9,0,0,0,3.2,0,320,0,0,0,10,3,0,13


In [ ]:
data1.to_csv("weather_data_day.csv", mode='w')

###### 버리기 아까운 혹시몰라서

In [ ]:
# data1 = data1.astype({3: np.float,4: np.float,5: np.float,6: np.float
#                       ,7: np.float,8: np.float,9: np.float
#                       ,10: np.float,11: np.float,12: np.float
#                       ,13: np.float,14: np.float,16: np.float})
# data1.dftype()    
# arr = ['date','tem','temQC'
#        ,'rain','rainQC'
#        ,'windSpeed','windSpeedQC','windDirection','windDirectionQC'
#        ,'snow','snow3Hour'
#        ,'cloud','middleCloud','clodeUn','highCloud']